# Lets Calibrate the Camera (Estimation of Intrincics)

In [17]:
import cv2
import numpy as np
import os

import cv2.aruco as aruco


squaresX = 31  # number of chessboard squares along the x-axis
squaresY = 16  # number of chessboard squares along the y-axis
squareLength = 0.04933  # 4.933 cm in meters
markerLength = 0.03846  # 3.846 cm in meters


aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
board_size = (squaresX, squaresY)
board = aruco.CharucoBoard(board_size, squareLength, markerLength, aruco_dict)
board.setLegacyPattern(True) 

from_cv_img = board.generateImage((2100, 1100))
# cv2.imshow("board_6_4_dict_0", from_cv_img)
cv2.imwrite("charuco_board.png", from_cv_img)



True

### Lets Load Right and Left Images

In [18]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Add other file extensions if needed.
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
    return images

# Example usage:
right_folder_path = "right/"
right_images = read_images_from_folder(right_folder_path)
left_folder_path = "left/"
left_images = read_images_from_folder(left_folder_path)


# If you want to display the images to check:
print(len(right_images))
print(len(left_images))



39
39


### 1. Detect Markers
### 2. Interpolate Corner using known Board detials to improve detection
### 3. Collect Corners and id of Markers.

In [19]:
all_corners = []
all_ids = []
for image in right_images:
    # Convert to grayscale for detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect ArUco markers
    markers, ids, _ = aruco.detectMarkers(gray, aruco_dict)
    
    # Draw markers on image for visual inspection
    image_with_markers = aruco.drawDetectedMarkers(image.copy(), markers)
    cv2.imshow('Detected ArUco markers', image_with_markers)
    cv2.waitKey(500)
    cv2.destroyAllWindows()

    
    if ids is not None:
        print(f"Detected {len(ids)} ArUco markers.")
        
        # Interpolate Charuco corners
        retval, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(markers, ids, gray, board)
        
        if charuco_corners is not None and charuco_ids is not None and len(charuco_corners) > 5:
            all_corners.append(charuco_corners)
            all_ids.append(charuco_ids)
            print(f"Detected {len(charuco_ids)} Interpolated ArUco markers.")
        else:
            print("ChArUco corners could not be interpolated.")
    else:
        print("No ArUco markers detected.")

print(len(all_corners), "images have valid ChArUco corners detected.")
cv2.destroyAllWindows()



Detected 153 ArUco markers.
Detected 232 Interpolated ArUco markers.
Detected 153 ArUco markers.
Detected 245 Interpolated ArUco markers.
Detected 176 ArUco markers.
Detected 307 Interpolated ArUco markers.
Detected 146 ArUco markers.
Detected 257 Interpolated ArUco markers.
Detected 177 ArUco markers.
Detected 319 Interpolated ArUco markers.
Detected 168 ArUco markers.
Detected 292 Interpolated ArUco markers.
Detected 167 ArUco markers.
Detected 285 Interpolated ArUco markers.
Detected 148 ArUco markers.
Detected 247 Interpolated ArUco markers.
Detected 161 ArUco markers.
Detected 276 Interpolated ArUco markers.
Detected 138 ArUco markers.
Detected 235 Interpolated ArUco markers.
Detected 148 ArUco markers.
Detected 245 Interpolated ArUco markers.
Detected 138 ArUco markers.
Detected 239 Interpolated ArUco markers.
Detected 163 ArUco markers.
Detected 283 Interpolated ArUco markers.
Detected 158 ArUco markers.
Detected 282 Interpolated ArUco markers.
Detected 183 ArUco markers.
Detect

### Using Calbrate CameraCharuco estimate Intrincics (fx, fy, cx, cy), distortion parameters (Fisk eys model - k1, k2, k3, k4)

In [20]:
imsize = right_images[0].shape[:2]
fx = 670.0
fy = 670.0
cx = 640.0
cy = 480.0
   
initial_camera_matrix = np.array([
   [fx, 0, cx],
   [0, fy, cy],
   [0,  0,  1]
])

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
flags = 0
flags |= cv2.CALIB_USE_INTRINSIC_GUESS
flags |= cv2.CALIB_RATIONAL_MODEL

r_retval, r_cameraMatrix, r_distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
    charucoCorners=all_corners,
    charucoIds=all_ids,
    board=board,
    imageSize=imsize,
    cameraMatrix=initial_camera_matrix,
    distCoeffs=None,
    flags=flags,
    criteria=criteria
)


In [21]:
print(r_retval)

0.3576105489838216


In [22]:
print(r_cameraMatrix)

[[570.01370738   0.         644.79305327]
 [  0.         570.36462896 383.24616992]
 [  0.           0.           1.        ]]


In [23]:
print(r_distCoeffs)

[[ 3.04364691e-01 -3.39232891e-02  1.17107589e-04  3.01629606e-05
  -8.25181503e-04  6.34493910e-01 -1.26666746e-02 -6.54137066e-03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [24]:
for i in range(len(right_images)):
    rectified_img = cv2.undistort(right_images[i], r_cameraMatrix, r_distCoeffs,newCameraMatrix=None)
    cv2.imwrite("rec_right/"+str(i)+".png", rectified_img)
    i=i+1


In [25]:
all_corners = []
all_ids = []
for image in left_images:
    # Convert to grayscale for detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect ArUco markers
    markers, ids, _ = aruco.detectMarkers(gray, aruco_dict)
    
    # Draw markers on image for visual inspection
    image_with_markers = aruco.drawDetectedMarkers(image.copy(), markers)
    cv2.imshow('Detected ArUco markers', image_with_markers)
    cv2.waitKey(500)
    cv2.destroyAllWindows()

    
    if ids is not None:
        print(f"Detected {len(ids)} ArUco markers.")
        
        # Interpolate Charuco corners
        retval, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(markers, ids, gray, board)
        
        if charuco_corners is not None and charuco_ids is not None and len(charuco_corners) > 5:
            all_corners.append(charuco_corners)
            all_ids.append(charuco_ids)
            print(f"Detected {len(charuco_ids)} Interpolated ArUco markers.")
        else:
            print("ChArUco corners could not be interpolated.")
    else:
        print("No ArUco markers detected.")

print(len(all_corners), "images have valid ChArUco corners detected.")
cv2.destroyAllWindows()



Detected 145 ArUco markers.
Detected 226 Interpolated ArUco markers.
Detected 186 ArUco markers.
Detected 331 Interpolated ArUco markers.
Detected 150 ArUco markers.
Detected 261 Interpolated ArUco markers.
Detected 136 ArUco markers.
Detected 230 Interpolated ArUco markers.
Detected 177 ArUco markers.
Detected 312 Interpolated ArUco markers.
Detected 128 ArUco markers.
Detected 225 Interpolated ArUco markers.
Detected 155 ArUco markers.
Detected 272 Interpolated ArUco markers.
Detected 142 ArUco markers.
Detected 249 Interpolated ArUco markers.
Detected 184 ArUco markers.
Detected 319 Interpolated ArUco markers.
Detected 174 ArUco markers.
Detected 311 Interpolated ArUco markers.
Detected 153 ArUco markers.
Detected 270 Interpolated ArUco markers.
Detected 157 ArUco markers.
Detected 271 Interpolated ArUco markers.
Detected 161 ArUco markers.
Detected 264 Interpolated ArUco markers.
Detected 167 ArUco markers.
Detected 282 Interpolated ArUco markers.
Detected 180 ArUco markers.
Detect

In [26]:
imsize = right_images[0].shape[:2]
fx = 670.0
fy = 670.0
cx = 640.0
cy = 480.0
   
initial_camera_matrix = np.array([
   [fx, 0, cx],
   [0, fy, cy],
   [0,  0,  1]
])

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
flags = 0
flags |= cv2.CALIB_USE_INTRINSIC_GUESS
flags |= cv2.CALIB_RATIONAL_MODEL

l_retval, l_cameraMatrix, l_distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
    charucoCorners=all_corners,
    charucoIds=all_ids,
    board=board,
    imageSize=imsize,
    cameraMatrix=initial_camera_matrix,
    distCoeffs=None,
    flags=flags,
    criteria=criteria
)


In [27]:
print(l_retval)

0.3691704770007572


In [28]:
print(l_cameraMatrix)

[[573.82488599   0.         630.08758367]
 [  0.         573.84024759 382.12561572]
 [  0.           0.           1.        ]]


In [29]:
print(l_distCoeffs)

[[ 2.93269155e-01 -3.75668944e-02  1.02177640e-04  1.22342540e-04
  -1.04479176e-03  6.20933219e-01 -1.84024849e-02 -7.52657979e-03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [30]:
for i in range(len(left_images)):
    rectified_img = cv2.undistort(left_images[i], l_cameraMatrix, l_distCoeffs,newCameraMatrix=None)
    cv2.imwrite("rec_left/"+str(i)+".png", rectified_img)
    i=i+1


### Now we have calibrated both of Stereo Pair


In [31]:
print("Left Camera Matrix", l_cameraMatrix)

Left Camera Matrix [[573.82488599   0.         630.08758367]
 [  0.         573.84024759 382.12561572]
 [  0.           0.           1.        ]]


In [32]:
print("Right Camera Matrix", r_cameraMatrix)

Right Camera Matrix [[570.01370738   0.         644.79305327]
 [  0.         570.36462896 383.24616992]
 [  0.           0.           1.        ]]
